# Scraping Boxofficemojo

### Scraping studio links

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
import scraping_class
logfile = 'sds19exam2.csv'
connector = scraping_class.Connector(logfile)

response, my_id = connector.get('https://www.boxofficemojo.com/studio/', 'scraping-bom')
html = response.text
soup = bs(html,'lxml')

table_node = soup.find_all('table', attrs = {'border': '0', 'cellspacing': '1', 'cellpadding': '5'})
link_nodes = soup.find_all('a', href = True)
studios = []
for link in link_nodes:
    if '/studio/chart/' in link['href']:
        studios.append(link['href'].split('&studio=')[1].split('.htm')[0])

### Function for scraping a studio's movies

In [2]:
def get_movies(studio):
    i = 1
    while True:
        logfile = 'sds19exam2.csv'
        connector = scraping_class.Connector(logfile)
        response, my_id = connector.get('https://www.boxofficemojo.com/studio/chart/?page=' + str(i) + '&yr=&view=company&view2=allmovies&studio=' + studio + '.htm&p=.htm', 'scraping-bom')
        html = response.text
        tables = pd.read_html(html, header = 0)
        
        # OLD: tables = pd.read_html('https://www.boxofficemojo.com/studio/chart/?page=' + str(i) + '&yr=&view=company&view2=allmovies&studio=' + studio + '.htm&p=.htm', header = 0)
        if len(tables) == 2: break
        dfs = tables[1]
        dfs.drop(dfs.tail(4).index, inplace = True)
        ### TEMPORARY dfs.columns = ['boRank', 'title', 'studioAcronym', 'boWorldwide', 'totalTheaters', 'boOpening', 'openingTheaters', 'openDate'] 
        if i == 1:
            dfn = dfs.copy()
        else:
            dfn = dfn.append(dfs)
        i += 1
        time.sleep(1)
    try:
        return dfn
    except:
        return None

### Scraping all movies

In [ ]:
i = 1
for studio in studios:
    if i == 1:
        dff = get_movies(studio)
    else:
        try:
            dff = dff.append(get_movies(studio))
        except:
            pass
    print(i)
    i += 1
    time.sleep(1)
dff

C:\Users\Jacob\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


### Structuring data

In [ ]:
dff = pd.read_pickle('bom_raw.pkl')

In [ ]:
def convert_year(string):
    try:
        string = string.split('/')[2]
        if string[0] in '23456789':
            return int('19' + string)
        else:
            return int('20' + string)
    except:
        return None
    
def numerize_money(string):
    if isinstance(string, str):
        try:
            return int(float(string.replace('$', '').replace(',', '')))
        except:
            return None
    else:
        return None


dff['boOpening'] = dff['boOpening'].apply(numerize_money)
dff['boWorldwide'] = dff['boWorldwide'].apply(numerize_money)
dff['year'] = dff['openDate'].apply(convert_year)
dff['year'] = dff['openDate'].apply(convert_year)
dff = dff.drop('openDate', axis = 1)

############# TEMPORARY #############
### --> dff.to_pickle('bom_clean.pkl')
############# TEMPORARY #############

In [ ]:
dff